In [ ]:
import numpy as np

### 概率求解：给定观测序列和模型参数计算观测序列出现的概率
1. 前向算法：根据初值计算后一时刻观测值的概率
    - 初值
    $$\alpha(i)=pi_ib_i(o_1), i=1,2,3,...N$$
    - 递推 对t=1, 2, 3, ..., T - 1
    $$\alpha_{t+1} = \left[\sum_{j=1}^N \alpha_t(j)a_{ji} \right]b_i(o_{t+1}), i=1,2,...,N$$
    - 终止
    $$P(O|\lambda) = \sum_{i=1}^N \alpha_T(i)$$

In [ ]:
# 例 10.2（p200页）
# 状态集合Q = {1, 2, 3},观测集合V={红，白} 0 1
A = np.array([[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]])
B = np.array([[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]])
Pi = np.array([0.2, 0.4, 0.4])
V = ['红', '白']    # 观测值集合
# 设T = 3，观测值O = {红 白 红}
T = 3   # 时间
N = 3   # 隐藏状态数量
O = ['红', '白', '红', '白', '白']   # 观测值

In [ ]:
# 1. 计算初值
alpha1 = Pi * B[:, 0]   # 初始观测值为0时的概率

# 2. 递推计算
alpha2 = np.array([(alpha1 * A[:, i]).sum() * B[i, 1] for i in range(N)])
alpha3 = np.array([(alpha2 * A[:, i]).sum() * B[i, 0] for i in range(N)])
# alpha4 = np.array([(alpha3 * A[:, i]).sum() * B[i, 0] for i in range(N)])

# 3. 终止
result = alpha3.sum()
print(alpha1)
print(alpha2)
print(alpha3)
print(result)

In [ ]:
# 封装函数
def forward(obs, V, Pi, A, B):
    """ 前向算法计算概率值，知道当前时刻观测值的概率，计算下一个时刻观测值的概率"""
    T = len(obs)
    N = len(Pi)
    # 计算初值
    alpha1 = Pi * B[:, 0]
    # 递推计算
    def cal_alpha(alpha, o):
        return np.array([(alpha * A[:, i]).sum() * B[i, o] for i in range(N)])
    
    for j in obs[1:]:
        o = V.index(j)
        alpha = alpha1
        alpha1 =  cal_alpha(alpha, o)
    # 返回终值
    return alpha1.sum()

forward(O, V, Pi, A, B)

2. 后向算法：根据最后时刻观测值的概率计算前一个时刻观测值的概率
    - 初值
    $$\beta_T(i) = 1, i=1, 2, ..., N$$
    - 对t=T-1, T-2, ...,1
    $$\beta_t(i)=\sum_{j=1}^N a_{ij}b_j(o_{t+1}) \beta_{t+1}(j), i=1, 2, ...,N$$
    - 终值
    $$P(O|\lambda)=\sum_{i=1}^N \pi_i b_i(o_1) \beta_1(i)$$

In [ ]:
# 计算T时刻的不同状态下的概率
beta3 = [1, 1, 1]     # 每列代表不同的状态
# 求T-1，即2时刻的概率,状态1,
beta2 = np.array([(A[i, :] * B[:, 0] * beta3).sum() for i in range(N)])
beta1 = np.array([(A[i, :] * B[:, 1] * beta2).sum() for i in range(N)])
# 求终值
p = (Pi * B[:, 0] * beta1).sum()
print(p)

In [ ]:
def backward(obs, V, Pi, A, B):
    """ 后向算法 """
    # 得到相关参数
    T = len(obs)
    N = len(Pi)
    
    # 最终时刻的概率
    last_beta = np.ones((3, ))
    def  cal_result(beta, o):
        return np.array([(A[i, :] * B[:, o] * beta).sum() for i in range(N)])
    
    for j in obs[::-1][:-1]:
        o = V.index(j)    # t+1时刻的状态值
        beta = last_beta
        last_beta = cal_result(beta, o)
    return (Pi * B[:, 0] * last_beta).sum()

print(backward(O, V, Pi, A, B))

### 预测算法：又称解码算法，即给定观测序列，求解最可能的状态序列
1. 近似算法：在每个时刻选择在该时刻最可能出现的状态，从而得到一个状态序列作为结果
2. 维特比算法：用动态规划求解最有路径
    - 初始化：
    $$\delta_1(i) = \pi_i * b_i(o_1), \;\;i=1, 2, ..., N$$
    $$\Psi_1(i) = 0,\;\; i=1, 2, ..., N$$
    - 递推。对t=2, 3, ..., T
    $$\delta = \mathop{max}\limits_{1\leq j \leq N} [\delta_{t-1}(j) a_{ji}]b_i(o_t),\;i=1,2,...,N$$
    $$\Psi(i)=arg \mathop{max}\limits_{1\leq j \leq N}[\delta_{t-1}(j)a_{ji}],\; i=1,2,...,N$$
    - 终止
    $$P^* = \mathop{max}\limits_{1\leq j \leq N} \delta_T(i)$$
    $$i_T^* = arg \mathop{max}\limits_{1\leq j \leq N}[\delta_T(i)]$$
    - 最优路径回溯，对t=T-1,T-2,..., 1
    $$i_t^* = \Psi_{t+1}(i_{t+1}^*)$$
    最优路径$I^* = (i_1^*,\; i_2^*, \;..., \;i_T^*)$

In [ ]:
# 例 10.3（P210)
